In [2]:
import pandas as pd
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from math import sqrt
from sklearn.metrics import roc_auc_score, f1_score, classification_report, confusion_matrix
from matplotlib import pyplot
from sklearn.metrics import mean_squared_error, classification_report
from keras.models import Sequential, Model 
from keras.layers import Dense, Input, concatenate, Activation, Dropout
from keras.optimizers import Adam
import tensorflow
from keras.utils import to_categorical
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from micromlgen import port

Using TensorFlow backend.
C:\Users\luigi.borriello2\.conda\envs\ts\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\luigi.borriello2\.conda\envs\ts\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\luigi.borriello2\.conda\envs\ts\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\luigi.borriello2\.conda

In [3]:
RANDOM_SEED = 42

In [4]:
np.random.seed(RANDOM_SEED)
tensorflow.set_random_seed(RANDOM_SEED)

In [5]:
with open('data/X_paper.pkl', 'rb') as f:
    X = pickle.load(f)

with open('data/y_paper.pkl', 'rb') as f:
    y = pickle.load(f)

In [6]:
X.shape

(1350, 5)

In [7]:
X = np.concatenate((X[:250], X[750:1000], X[1500:]), axis=0)
y = np.concatenate((y[:250], y[750:1000], y[1500:]), axis=0)

Split in train and test sets

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=RANDOM_SEED)

In [8]:
#y_train = pd.get_dummies(y_train).values
#y_test = pd.get_dummies(y_test).values

In [9]:
y_train

array([6, 6, 0, 0, 6, 3, 0, 6, 6, 3, 3, 3, 3, 3, 0, 3, 3, 3, 0, 3, 3, 3,
       0, 0, 6, 0, 6, 3, 3, 0, 3, 0, 3, 6, 3, 6, 0, 3, 0, 6, 6, 6, 0, 0,
       6, 0, 6, 0, 3, 6, 0, 0, 0, 0, 0, 0, 0, 3, 3, 6, 3, 0, 6, 3, 3, 3,
       0, 6, 6, 3, 0, 0, 6, 3, 6, 0, 3, 0, 0, 3, 0, 6, 0, 3, 6, 6, 0, 0,
       0, 0, 3, 0, 0, 0, 3, 3, 0, 6, 0, 3, 3, 0, 0, 0, 3, 0, 3, 0, 6, 3,
       3, 0, 3, 0, 3, 3, 3, 0, 3, 6, 0, 3, 3, 3, 3, 6, 0, 4, 3, 0, 0, 3,
       3, 6, 6, 0, 0, 6, 3, 6, 6, 0, 3, 0, 3, 6, 0, 0, 3, 3, 6, 6, 0, 6,
       3, 0, 3, 3, 6, 6, 0, 0, 3, 0, 0, 0, 6, 0, 0, 3, 0, 6, 6, 3, 0, 0,
       6, 0, 3, 3, 6, 3, 3, 0, 4, 6, 3, 6, 6, 0, 6, 0, 3, 3, 3, 6, 0, 3,
       3, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 3, 3, 0, 0, 4, 0, 0, 3, 0, 3, 0,
       0, 0, 0, 3, 0, 3, 0, 3, 3, 6, 0, 3, 3, 6, 0, 0, 0, 6, 3, 4, 0, 6,
       3, 3, 6, 3, 6, 6, 6, 0, 3, 0, 3, 0, 6, 6, 0, 6, 0, 3, 0, 3, 6, 6,
       3, 6, 3, 0, 6, 3, 3, 3, 0, 0, 0, 3, 3, 3, 6, 6, 6, 3, 3, 0, 6, 0,
       6, 0, 3, 3, 0, 3, 0, 0, 0, 6, 3, 0, 6, 3, 6,

In [10]:
X_train.shape[1]

5

## Models Evaluation

### Random Forest

In [11]:
model = RandomForestClassifier(n_estimators=50, random_state=42)
#model = MLPClassifier(random_state=42)

In [12]:
model.fit(X_train, y_train)

RandomForestClassifier(n_estimators=50, random_state=42)

In [13]:
y_pred = model.predict(X_test)

In [14]:
f1_score(y_test, y_pred, average="weighted")

0.9662605435801312

In [15]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        50
           3       0.94      1.00      0.97        47
           4       0.00      0.00      0.00         3
           6       1.00      1.00      1.00        32

    accuracy                           0.98       132
   macro avg       0.73      0.75      0.74       132
weighted avg       0.96      0.98      0.97       132



C:\Users\luigi.borriello2\.conda\envs\ts\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\luigi.borriello2\.conda\envs\ts\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\luigi.borriello2\.conda\envs\ts\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [16]:
print(confusion_matrix(y_test, y_pred))

[[50  0  0  0]
 [ 0 47  0  0]
 [ 0  3  0  0]
 [ 0  0  0 32]]


# Export the model and save it on File

In [17]:
with open('rf_model_paper_3.h', 'w') as f:  # change path if needed
    f.write(port(model, optimize=False))

In [18]:
with open('models/rf_model_paper_3.pkl', 'wb') as f:
    pickle.dump(model, f)
